# Чистовой вариант

Я работал сразу в нескольких нотбуках - я писал функции, которые строят большое количество моделей с разными параметрами и оценивают их, такие функции работают долго, поэтому запустив функцию я создавал копию нотбука и продолжал работать в копии.

В этих функциях я использовал print - чтобы видеть результат как только он вычислился без необходимости ждать, когда все модели посчитаются. Чтобы не загромождать нотбук, я переносил результаты в Эксель файл и очищал вывод.

В Экселе и сравнивал результаты. Графики я не делал, можно сказать не успел - мне было проще в табличном виде смотреть, поэтому заранее всё не построил =(.

Чистовой нотбук называется Building model.ipynb - в нём строится одна модель с теми параметрами, по которым у меня получился максимальный r2.

Много интересного в нотбуке Building model - Forests.ipynb - там я подбирали итоговые параметры, которые и вошли в чистовой вариант.

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100

### 1) Читаем данные

In [2]:
data = pd.read_csv('input/train.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [4]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


### 2) Разбиваем на трейн и валид

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, valid = train_test_split(data, test_size=0.3, random_state=100)

In [7]:
train.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price'],
      dtype='object')

### 3) Пишем функции для обработки данных

In [8]:
def clean_kitchen(df):
    df.loc[df['KitchenSquare'] > df['Square'] * 0.4, 'KitchenSquare'] = df['Square'] * 0.15
    return df

In [9]:
def clean_rooms(df):
    df.loc[df['Rooms'] > 5, 'Rooms'] = 5
    return df

In [10]:
def clean_square(df):
    df.loc[df['Square'] < 15, 'Square'] = 15
    df['LifeSquare'] = df['LifeSquare'].fillna(df['Square'] - df['KitchenSquare'])
    df.loc[df['Square'] < df['LifeSquare'] + df['KitchenSquare'], 'Square'] = df['LifeSquare'] + df['KitchenSquare']
    return df

In [11]:
def clean_year(df):
    df.loc[df['HouseYear'] > 2020, 'HouseYear'] = 2020
    return df

In [12]:
stat0 = train.groupby(['DistrictId', 'Rooms', 'Social_2'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price_drs'})
stat1 = train.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price_dr'})
stat2 = train.groupby(['Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price_r'})
mean_price = train['Price'].mean()

In [13]:
def add_mean_price(df, stat0, stat1, stat2, mean_price):
    df = pd.merge(df, stat0, on=['DistrictId', 'Rooms', 'Social_2'], how='left')
    df = pd.merge(df, stat1, on=['DistrictId', 'Rooms'], how='left')
    df = pd.merge(df, stat2, on=['Rooms'], how='left')
    df['mean_price_r'] = df['mean_price_r'].fillna(mean_price)
    df['mean_price_dr'] = df['mean_price_dr'].fillna(df['mean_price_r'])
    df['mean_price_drs'] = df['mean_price_drs'].fillna(df['mean_price_dr'])
    return df

In [14]:
def do_label(df, feature):
    lab_enc = preprocessing.LabelEncoder()
    df.loc[:, feature] = lab_enc.fit_transform(df.loc[:, feature])
    return df
def dummies_handling(df):
    df = pd.get_dummies(df, columns = ['Ecology_2', 'Ecology_3', 'Shops_2'])
    #df = do_label(df, 'Social_2')
    return df

In [15]:
def floors_checking(df):
    df['HouseFloor'] = df['HouseFloor'].astype(int)
    df.loc[df['HouseFloor'] <= 0, 'HouseFloor'] = df['Floor'] + 1 # Не на последнем
    to_change = df['Floor'] > df['HouseFloor']
    df.loc[to_change, 'Floor'], df.loc[to_change, 'HouseFloor'] = df.loc[to_change, 'HouseFloor'], df.loc[to_change,'Floor']
    df['LastFloor'] = 0
    df.loc[df['Floor'] == df['HouseFloor'], 'LastFloor'] = 1
    df.loc[df['Floor'] == 1, 'LastFloor'] = 1
    return df

In [16]:
hc_stat1 = train.groupby(['DistrictId'], as_index=False)[['Healthcare_1']].mean().rename(columns={'Healthcare_1':'mean_hc_1'})
mean_hc_1 = train['Healthcare_1'].mean()

In [17]:
def clean_healthcare(df, hc_stat1, mean_hc_1):
    df['Helthcare_2'] = df['Helthcare_2'].astype(int)
    df = pd.merge(df, hc_stat1, on=['DistrictId'], how='left')
    df['mean_hc_1'] = df['mean_hc_1'].fillna(mean_hc_1)
    df['Healthcare_1'] = df['Healthcare_1'].fillna(df['mean_hc_1'])    
    return df

In [18]:
def prepare_data(df, stat1=stat1, stat2=stat2, mean_price=mean_price, hc_stat1=hc_stat1, mean_hc_1=mean_hc_1):
    df = clean_kitchen(df)
    df = clean_rooms(df)
    df = clean_square(df)
    df = clean_year(df)
    df = dummies_handling(df)
    df = add_mean_price(df, stat0, stat1, stat2, mean_price)
    df = floors_checking(df)
    df = clean_healthcare(df, hc_stat1, mean_hc_1)
    return df

### 4) Чистка данных

Я решил не удалять строки - всё равно из Теста их нельзя удалять

### 5) Применяем функции на трейне и валиде

In [19]:
train = prepare_data(train)
valid = prepare_data(valid)

In [20]:
train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B,mean_price_drs,mean_price_dr,mean_price_r,LastFloor,mean_hc_1
0,13182,1,3.0,81.107658,80.107658,1.0,10,17,1977,0.007122,1,264,0,228.292683,0,1,250381.580780,0,1,0,1,0,1,249368.226510,242920.318313,288183.829856,0,228.292683
1,10195,27,3.0,50.841082,34.225561,8.0,7,12,2013,0.072158,2,629,1,1140.695855,0,0,143545.018507,0,1,0,1,1,0,211930.719662,210532.678647,288183.829856,0,1140.695855
2,9055,53,2.0,52.301049,33.105796,8.0,4,12,1975,0.049637,34,7759,0,229.000000,1,3,270794.714055,0,1,0,1,0,1,261526.595768,257896.225214,214622.517478,0,746.975806
3,2819,52,2.0,68.935850,68.935850,0.0,16,17,1977,0.371149,34,7065,1,750.000000,2,5,168600.632600,0,1,0,1,0,1,177466.581911,177466.581911,214622.517478,0,750.000000
4,16504,27,1.0,78.944462,58.944462,20.0,6,6,2017,0.075424,11,3097,0,1140.695855,0,0,195520.319385,0,1,0,1,0,1,136591.260374,121865.415028,159775.950796,1,1140.695855


In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 28 columns):
Id                7000 non-null int64
DistrictId        7000 non-null int64
Rooms             7000 non-null float64
Square            7000 non-null float64
LifeSquare        7000 non-null float64
KitchenSquare     7000 non-null float64
Floor             7000 non-null int64
HouseFloor        7000 non-null int64
HouseYear         7000 non-null int64
Ecology_1         7000 non-null float64
Social_1          7000 non-null int64
Social_2          7000 non-null int64
Social_3          7000 non-null int64
Healthcare_1      7000 non-null float64
Helthcare_2       7000 non-null int32
Shops_1           7000 non-null int64
Price             7000 non-null float64
Ecology_2_A       7000 non-null uint8
Ecology_2_B       7000 non-null uint8
Ecology_3_A       7000 non-null uint8
Ecology_3_B       7000 non-null uint8
Shops_2_A         7000 non-null uint8
Shops_2_B         7000 non-null uint8
mean_

### 6) Строим модель, делаем предсказание на трейне и валиде

In [22]:
from sklearn.ensemble import RandomForestRegressor as RF

In [23]:
model = RF(n_estimators=214, max_depth=17, random_state=100)

In [24]:
feats = ['Rooms', 'Square', 'Helthcare_2', 'Shops_1', 'Shops_2_A',\
         'Social_1', 'Social_3', 'Ecology_1',\
         'KitchenSquare', 'HouseFloor', 'Floor', 'HouseYear', 'Healthcare_1']

In [25]:
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=17,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=214, n_jobs=1,
           oob_score=False, random_state=100, verbose=0, warm_start=False)

### 7) Проверяем качество модели

In [26]:
from sklearn.metrics import r2_score

In [27]:
def pred_evaluate_model(df, model=model, feats=feats, calculate_r2='Yes'):
    pred = model.predict(df.loc[:, feats])
    if calculate_r2 == 'Yes':
        r2 = r2_score(df['Price'], pred)
        print(f'R2: {r2}')
    return pred

In [28]:
def evaluate_model(df, model=model, feats=feats):
    pred = model.predict(df.loc[:, feats])
    return r2_score(df['Price'], pred)

In [29]:
pred_train = pred_evaluate_model(train)

R2: 0.9487078496535671


In [30]:
pred_valid = pred_evaluate_model(valid)

R2: 0.7247925782368756


### 8) Применяем функции обработки данных, чистим тестовые данные

In [31]:
test = pd.read_csv('input/test.csv')
test = prepare_data(test)

### 9) Делаем предсказание на тесте

In [32]:
test['Price'] = pred_evaluate_model(test, calculate_r2='No')

### 10) Записываем предсказания в файл

In [33]:
test.loc[:, ['Id', 'Price']].to_csv('VVashchenkov_predictions.csv', index=None)